In [1]:
# !capture --no-stderr
# !pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python 

In [2]:
# !pip install "nomic[local]"

In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("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")
_set_env("nk-p4RbLXYiBQInfAQyrlCssat_n9w-697uXrq4dlCmq0o")

**Libraries**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_nomic.embeddings import NomicEmbeddings

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain import hub
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
import nest_asyncio
nest_asyncio.apply()
from langgraph.graph import END, StateGraph, START

from llama_cloud_services import LlamaParse
from llama_index.core import Document as LlamaDocument
from llama_index.core import SimpleDirectoryReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dataclasses import dataclass

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from langchain_nomic.embeddings import NomicEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [24]:
# from llama_cloud_services import LlamaParse
# from llama_index.core import Document as LlamaDocument
# from llama_index.core import SimpleDirectoryReader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from dataclasses import dataclass
# from langchain_community.vectorstores.utils import filter_complex_metadata


# LLAMA_CLOUD_API_KEY="llx-nxrF5SQ5MnbMrKhrb1HqfdFA3YajxTDjtjfctsyebDSdBa7W"

# @dataclass
# class Document:
#     """Document class compatible with LangChain"""
#     page_content: str
#     metadata: dict = None

#     def __post_init__(self):
#         if self.metadata is None:
#             self.metadata = {}

# def load_and_process_documents(file_paths):
#     """Load and process PDF documents using LlamaParse"""
#     try:
#         # Configure parser
#         parser = LlamaParse(
#         result_type="markdown",
#         api_key=LLAMA_CLOUD_API_KEY
#         )        
#         file_extractor = {".pdf": parser}
        
#         # Load raw documents
#         raw_docs = SimpleDirectoryReader(
#             input_files=file_paths,
#             file_extractor=file_extractor
#         ).load_data()
        
#         # Convert to LangChain documents
#         documents = [
#             Document(
#                 page_content=doc.text,
#                 metadata={"source": file_paths}
#             ) for doc in raw_docs
#         ]
        
#         return documents
                
#     except Exception as e:
#         print(f"Error processing documents: {str(e)}")
#         raise


In [35]:
# LLAMA_CLOUD_API_KEY="llx-nxrF5SQ5MnbMrKhrb1HqfdFA3YajxTDjtjfctsyebDSdBa7W"

# def process_pdf(file_path):
#     try:
#         # Configure LlamaParse
#         parser = LlamaParse(result_type="markdown",api_key=LLAMA_CLOUD_API_KEY)
#         file_extractor = {".pdf": parser}
        
#         # Load document
#         raw_docs = SimpleDirectoryReader(
#             input_files=[file_path],
#             file_extractor=file_extractor
#         ).load_data()
        
#         # Convert to LangChain format
#         documents = [
#             Document(
#                 page_content=doc.text,
#                 metadata={"source": file_path}
#             ) for doc in raw_docs
#         ]
        
#         # Split text
#         text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#             chunk_size=1000,
#             chunk_overlap=5
#         )
#         doc_splits = text_splitter.split_documents(documents)
        
#         # Store in VectorDB
#         vectorstore = Chroma.from_documents(
#             documents=doc_splits,
#             collection_name="rag-chroma",
#             embedding=NomicEmbeddings(
#                 model="nomic-embed-text-v1.5",
#                 inference_mode="local"
#             ),
#         )
#         return vectorstore
#     except Exception as e:
#         print(f"Error processing PDF: {str(e)}")
#         raise

In [36]:
# from llama_index.vector_stores.qdrant import QdrantVectorStore
# from llama_index.core import Settings
# import qdrant_client
# from IPython.display import Markdown, display
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from llama_index.core import StorageContext
# from langchain_nomic.embeddings import NomicEmbeddings

In [ ]:
# local_files = ["/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf"]
# retriever = process_pdf(local_files).as_retriever()


In [ ]:
# local_files = ["/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf"]
# documents = load_and_process_documents(local_files)
# print(f"Generated {len(documents)} chunks")

Started parsing the file under job_id f4365c32-46c8-4685-bd18-86f7acfc041f
Generated 11 chunks


In [45]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the paths to your local files
local_files = [
    "/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf",
]

# Class to wrap the file content
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata or {}

# Function to load the content from local PDF files
def load_local_pdf(file_path):
    doc = fitz.open(file_path)
    content = ""
    for page in doc:
        content += page.get_text()
    return Document(content, metadata={"source": file_path})

# Load the documents from the local files
docs = [load_local_pdf(file_path) for file_path in local_files]

# Split the documents using the text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs)

In [46]:
doc_splits

[Document(metadata={'source': '/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf'}, page_content='Citation: Yaseen; Kwon, O.-J.; Kim, J.;\nJamil, S.; Lee, J.; Ullah, F. Next-Gen\nDynamic Hand Gesture Recognition:\nMediaPipe, Inception-v3 and\nLSTM-Based Enhanced Deep\nLearning Model. Electronics 2024, 13,\n3233. https://doi.org/10.3390/\nelectronics13163233\nAcademic Editors: Fath U Min Ullah,\nEstefanía Talavera and Nuno\nGonçalves\nReceived: 12 July 2024\nRevised: 6 August 2024\nAccepted: 12 August 2024\nPublished: 15 August 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder\nthe\nterms\nand\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nelectronics\nArticle\nNext-Gen Dynamic Hand Gesture Recognition: MediaPipe,\nInception-v3 and LSTM-Based Enhanced Deep Learning Model\nYaseen 1\n, Oh-Jin Kwon 1,*\n, Jaeho Kim 2\n, Sonain Jami

**VectorDB**

In [47]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)
retriever = vectorstore.as_retriever()

**Retrieval Grader**

In [48]:
local_llm = "llama3.1"

In [49]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "llm"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'no'}


In [50]:
docs

[Document(metadata={'source': '/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf'}, page_content='given number of test instances (1).\n•\nRecall: Also called true positive rate as it measures the model’s ability regarding true\npositives (2).\n•\nF1-score: Reflects the model’s ability concerning false positives and negatives (3).\n•\nSpecificity: Reflects the model’s ability to correctly identify true negative instances (4).\nFormulas for each of these metrics are given below:\nAccuracy =\nTP + TN\nTP + TN + FP + FN\n(1)\nRecall =\nTP\nTP + FN\n(2)\nF1-score = 2 × Precision × Recall\nPrecision + Recall\n(3)\nSpecificity =\nTN\nTN + FP\n(4)\nElectronics 2024, 13, 3233\n6 of 11\nResults based on the above metrics are shown in Table 1. For more visually appealing\nresults and deeper insights, we have used a confusion matrix as shown in Figures 3 and 4.\nAlso, to further evaluate our results for the proposed architecture, we replace the\nLSTM classifier with a gated recurrent unit (GRU

**Generate**

In [51]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])


# LLM
llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/home/neutrino/miniconda3/envs/Ml/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


It appears that you've provided a snippet of text from a research paper or article. The question is not explicitly stated, but I can try to infer what you might be looking for.

Based on the content, it seems like you're interested in understanding the proposed changes and improvements made to a baseline architecture for hand gesture recognition using deep learning models. Specifically, you might want to know about:

1. The use of MediaPipe as an ROI (Region of Interest) extractor.
2. The comparison with Inception v3 alone.
3. The reduction in computation complexity and cost.
4. The proposed hybrid model consisting of three main layers: MediaPipe for ROI extraction, Inception v3 as a feature extractor, and LSTM for classification.

If that's correct, I can try to provide a concise summary or answer your questions based on the provided text!


**Hallucination Grader**

In [52]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. \n 
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}
    Give a binary score 'yes' or 'no' score to indicate whether the answer is grounded in / supported by a set of facts. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

**Answer Grader**

In [53]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is useful to resolve a question. \n 
    Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question}
    Give a binary score 'yes' or 'no' to indicate whether the answer is useful to resolve a question. \n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

**Question Re-writer**

In [54]:
llm = ChatOllama(model=local_llm, temperature=0)

# Prompt
re_write_prompt = PromptTemplate(
    template="""You a question re-writer that converts an input question to a better version that is optimized \n 
     for vectorstore retrieval. Look at the initial and formulate an improved question. \n
     Here is the initial question: \n\n {question}. Improved question with no preamble: \n """,
    input_variables=["generation", "question"],
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'It seems like you want me to rewrite the initial question into a more optimized form for vector store retrieval, but the initial question itself isn\'t provided.\n\nHowever, I can guide you on how to improve a question for better vector store retrieval:\n\n1. **Remove Preamble**: If your question starts with phrases like "I was wondering," "Can someone help me understand," or similar, remove them as they don\'t add value to the query and might reduce its effectiveness in search.\n\n2. **Focus on Key Entities**: Identify key entities (people, places, things) that are crucial for understanding the question. These should be included in your revised question.\n\n3. **Use Specific Keywords**: Incorporate specific keywords related to the topic or subject of interest. This can help in retrieving more relevant information from a vector store.\n\n4. **Simplify and Clarify**: Simplify the language used while maintaining clarity. Avoid ambiguity by making sure what you\'re asking is clear.\n\n5.

# Main Workflow

In [55]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [56]:
### Nodes

def retrieve(state):
    # Retrieve documents

    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}


def generate(state):
    # Generate answer

    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):

    # Determines whether the retrieved documents are relevant to the question.


    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    # Transform the query to produce a better question.


    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}




def decide_to_generate(state):
    # Determines whether to generate an answer, or re-generate a question.


    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:

        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    # Determines whether the generation is grounded in the document and answers question.

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [57]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [58]:
from pprint import pprint

# Run
inputs = {"question": "Summarise the key findings of the paper."}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        print(f"Node '{key}':")
        pprint(value)
        print("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE---
Node 'retrieve':
{'documents': [Document(metadata={'source': '/mnt/MainDrive/Codes/Heckeer/electronics-13-03233.pdf'}, page_content='Data Brief 2022, 45, 108659. [CrossRef]\n37.\nDhingra, N.; Kunz, A. Res3atn-Deep 3D residual attention network for hand gesture recognition in videos. In Proceedings of the\n2019 International Conference on 3D Vision (3DV), Quebec City, QC, Canada, 16–19 September 2019; pp. 491–501.\nElectronics 2024, 13, 3233\n11 of 11\n38.\nSimonyan, K.; Zisserman, A. Two-stream convolutional networks for action recognition in videos. In Proceedings of the Advances\nin Neural Information Processing Systems, Montreal, QC, Canada, 8–13 December 2014; Volume 27.\n39.\nTran, D.; Bourdev, L.; Fergus, R.; Torresani, L.; Paluri, M. Learning spatiotemporal features with 3D convolutional networks. In\nProceedings of the IEEE International Conference on Computer Vision 2015, Santiago, Chile, 7–13 December 2015; pp. 4489–4497.\nDisclaimer/Publisher’s Note: The state